## Mask Detection with MobileNet

In [1]:
# Load the TensorBoard notebook extension:
%load_ext tensorboard

In [2]:
# Import libraries:

import shutil
import datetime
import tensorflow as tf
import cv2 
import os
import numpy as np
from keras.models import Model
from tqdm import tqdm_notebook as tqdm
from PIL import Image
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam, SGD
from focal_loss import BinaryFocalLoss
from sklearn.utils import shuffle

In [3]:
# Make use of TensorBoard:

try:
    shutil.rmtree('logs')
except:
    pass
try:
    shutil.rmtree('logsx')
except:
    pass

# Try with TB:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [4]:
# Import Image Dataset from local:

def read_images(path):
    files = os.listdir(path)
    files = [file for file in files if file[-4:] == ".jpg"]
        
    images = []
    for file in tqdm(files):
        try:
            image = Image.open(os.path.join(path, file))

            # https://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.resize
            image = image.resize((224, 224), Image.LANCZOS)

            # https://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.convert
            image = image.convert("RGB")

            image = np.asarray(image)

            images.append(image)
        except OSError:
            pass
    
    return images
    
with_mask = read_images(os.path.join("./dataset/cropped/", "with"))
without_mask = read_images(os.path.join("./dataset/cropped/", "without"))

C:\Users\Slavka\AppData\Local\Temp\ipykernel_22980\2666920087.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tqdm(files):


  0%|          | 0/2332 [00:00<?, ?it/s]

C:\Users\Slavka\AppData\Local\Temp\ipykernel_22980\2666920087.py:13: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image = image.resize((224, 224), Image.LANCZOS)


  0%|          | 0/1330 [00:00<?, ?it/s]

In [5]:
# Prepare Image Data in Data X and Target Values y:

with_mask = np.asarray(with_mask)
without_mask = np.asarray(without_mask)

X = np.concatenate([with_mask, without_mask])

y_with_mask = np.zeros(len(with_mask))
y_without_mask = np.ones(len(without_mask))

y = np.concatenate([y_with_mask, y_without_mask])

# Delete to free RAM:
del with_mask, without_mask, y_with_mask, y_without_mask

## Transfer Learning with MobileNet

In [6]:
# Use MobileNet Model for Transfer-Learning with non-trainable layers for faster processing:

mobilenet_model = MobileNet(include_top=False, input_shape=(224, 224, 3))
for layer in mobilenet_model.layers:
    layer.trainable = False

classifier_network = mobilenet_model.output
classifier_network = Flatten(name = "Flatten_Final")(classifier_network)
classifier_network = Dense(128, activation="relu",name = "Dense_Final")(classifier_network)
classifier_network = Dropout(0.5,name = "Dropout_Final")(classifier_network)
classifier_network = Dense(1, activation="sigmoid", name = "Sigmoid_Classifier" )(classifier_network)
mask_classifier = Model(inputs=mobilenet_model.input, outputs = classifier_network)


In [7]:
# Set the Hyper-Parameters

alpha = 0.00001
n_epochs = 10
mini_batch_size = 32
     

In [8]:
# Compile the Model

opt = Adam(learning_rate=alpha, decay = alpha/n_epochs)
mask_classifier.compile(optimizer=opt,loss=BinaryFocalLoss(gamma=2),metrics = ['accuracy'])
#mask_classifier.summary() 

In [9]:
# Use Preprocessing of keras MobileNet Library and shuffle Dataset afterwards for better Predictions:

X = preprocess_input(X)
X, y = shuffle(X, y)

In [10]:
# Train the Model

mask_classifier.fit(X,y, epochs=25, batch_size=32, validation_split=0.2, shuffle=True, callbacks=[tensorboard_callback])

Epoch 1/25
92/92 [==============================] - 7s 37ms/step - loss: 0.2512 - accuracy: 0.8156 - val_loss: 0.0572 - val_accuracy: 0.9400
Epoch 2/25
92/92 [==============================] - 3s 32ms/step - loss: 0.0581 - accuracy: 0.9344 - val_loss: 0.0447 - val_accuracy: 0.9509
Epoch 3/25
92/92 [==============================] - 3s 31ms/step - loss: 0.0367 - accuracy: 0.9512 - val_loss: 0.0356 - val_accuracy: 0.9577
Epoch 4/25
92/92 [==============================] - 3s 32ms/step - loss: 0.0229 - accuracy: 0.9665 - val_loss: 0.0312 - val_accuracy: 0.9604
Epoch 5/25
92/92 [==============================] - 3s 31ms/step - loss: 0.0155 - accuracy: 0.9758 - val_loss: 0.0291 - val_accuracy: 0.9659
Epoch 6/25
92/92 [==============================] - 3s 31ms/step - loss: 0.0120 - accuracy: 0.9833 - val_loss: 0.0313 - val_accuracy: 0.9659
Epoch 7/25
92/92 [==============================] - 3s 31ms/step - loss: 0.0102 - accuracy: 0.9860 - val_loss: 0.0298 - val_accuracy: 0.9714
Epoch 8/25
92

In [11]:
# And finally save the Model:

mask_classifier.save("./model/mobilenet.h5")